In [1]:
## Import relevant libraries
import sys
import glob

sys.path.append('../../py_files/')
import quadrop2 as qd

qd.set_plotting_style()

### Data pre-procesing

In [ ]:
# Example usage
base_dir = "../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/061525-ThTr-accesoryProteins/"
qd.consolidate_images(base_dir)

In [ ]:
# Example usage
data_path = "../../../../Thomson Lab Dropbox/David Larios/activedrops/main/061525-ThTr-accesoryProteins/2ulTM-0p2ulThTr-0p6else_/"

calibration_curve_paths = sorted(glob.glob("../../../../Thomson Lab Dropbox/David Larios/activedrops/calibration_curve/***ugml.tif"))


conditions_dict = {
    "ThTr": "Pos0", 
    "ThTr_T7": "Pos1", 
    "ThTr_PRC1": "Pos2", 
    "ThTr_HeAl": "Pos3", 
    "ThTr_FtsZ": "Pos4", 
    "ThTr_T7_ZapA": "Pos5", 
    "ThTr_T7_ZapA_FtsA_FtsZ": "Pos6", 
    "ThTr_CetZ": "Pos7", 
    "ThTr_Tau": "Pos8", 
    "ThTr_K401": "Pos9", 

}

# Organize PosX folders into condition folders  
qd.organize_conditions(data_path, conditions_dict)

# Now run the existing functions to reorganize the tiffs and rename the folders
conditions, subconditions = qd.prepare_conditions(data_path)
time_interval_list = [60] * len(conditions)  # time intervals in seconds between frames for each condition

# subconditions = ['Rep1']
print("Conditions:", conditions)
print("Subconditions:", subconditions)



In [ ]:
qd.reorgTiffsToOriginal(data_path, conditions, subconditions)


### Generate movies

In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions,    
    subconditions, 
    channel='cy5', 
    time_interval_list=time_interval_list, 
    vmax=18, 
    skip_frames=1, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5',  
    frame_rate=120,
    skip_frames=1
    )


In [ ]:
conditions

In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='cy5', 
    grid_rows=2, 
    grid_cols=5, 
    frame_rate=120,
    batch_size=50
    )


In [ ]:
# Call the function
qd.fluorescence_heatmap(
    data_path, 
    conditions[:], 
    subconditions, 
    channel='GFP', 
    time_interval_list=time_interval_list, 
    vmax=400, 
    skip_frames=16, 
    calibration_curve_paths=calibration_curve_paths, 
    show_scalebar=False,
    )

In [ ]:
# Example usage
qd.create_movies(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    frame_rate=60,
    skip_frames=1
    )


In [ ]:
qd.create_combined_heatmap_movie_custom_grid(
    data_path, 
    conditions, 
    subconditions, 
    channel='GFP', 
    grid_rows=2, 
    grid_cols=5,       
    frame_rate=7.5,
    batch_size=50
    )


In [ ]:
# qd.delete_temporary_image_directories(data_path, conditions, subconditions)

### Fluorescence Quantification

In [ ]:
conditions

In [ ]:
# Example usage
# mw_kda_list = [44.95] * len(conditions)
droplet_volume_list = [2] * len(conditions)
# protein_lengths_list = [401] * len(conditions) # last one is negative

qd.quantify_tiffiles(
    data_path, 
    conditions, 
    subconditions, 
    calibration_curve_paths, 
    droplet_volume_list, 
    time_interval_list, 
    skip_frames=1,

)

### PIV pre-processing

In [ ]:
conditions[1:2:]

In [ ]:
qd.split_tiffs(data_path, conditions, subconditions, channel='cy5', file_interval=1)



### PIV

In [ ]:
# Define feature limits and other parameters
v = 2E-7
velocity_limits = (0, v)
other_limits = (-0.0005, 0.0005)
skip_frames = 1 ### CHANGE THIS TO SKIP FRAMES


velocity_limits = (None, None)
other_limits = (None, None)


feature_limits = {
    # 'u [m/s]': (-v, v), 
    # 'v [m/s]': (-v, v), 
    # 'data type [-]': (None, None),
    'velocity magnitude [m/s]': velocity_limits,
    'vorticity [1/s]': other_limits,
    'divergence [1/s]': other_limits,
    # 'dcev [1]': (0, 250),
    'shear [1/s]': other_limits,
    'strain [1/s]': other_limits,
    'vector direction [degrees]': (-180, 180),
}


# Features for PCA and plotting
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [um/s]",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [um]", 
    "power [W]_mean",
    "work [J]",
]


In [ ]:
conditions[::]

In [ ]:
time_interval_list = [30*12, 30*18, 30*18, 30*18, 30*18, 30*18, 30*18, 30*18,  ]  # time intervals in seconds between frames for each condition


# Process PIV data
qd.process_piv_data(
    data_path, 
    conditions,    
    subconditions, 
    feature_limits, 
    time_interval_list, 
    min_frame=0, 
    max_frame=None, 
    skip_frames=1, 
    plot_autocorrelation=False, 
    frame_rate=1, 
    heatmaps=False
    )


In [ ]:
# Plot features and PCA
qd.plot_PIV_all(
    data_path, 
    conditions[:5:-1],
    subconditions, 
    features_pca, 
    min_frame=0, 
    max_frame=None
    )


### Expression + PIV

In [ ]:
# Combine the data and save it to the "output_data" directory
qd.combine_averaged_dataframes(data_path, conditions, subconditions)


In [ ]:
qd.merge_expression_piv_data(data_path)


In [ ]:
# x_column = "time (h)"  # Example x-axis column
# y_column = "Protein Concentration_nM"  # Example y-axis column


# x_column = "Protein Concentration_nM"  # Example y-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "velocity magnitude [m/s]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "power [W]_mean"  # Example y-axis column

# x_column = "time (h)"  # Example x-axis column
# y_column = "work [J]_mean"  # Example y-axis column

x_column = "time (h)"  # Example x-axis column
y_column = "distance [m]_mean"  # Example y-axis column


qd.plot_expression_piv(
    data_path,
    conditions,
    x_column, 
    y_column, 
    sigma_x=0.1, 
    sigma_y=10, 
    x_log=False, 
    y_log=True, 
    min_frame=0, 
    max_frame=None, 
    individual_plots=False
    )

In [ ]:

# List of features for PCA
features_pca = [
    "vorticity [1/s]_mean",
    "velocity magnitude [m/s]_mean",
    "distance [m]_mean",
    "divergence [1/s]_mean",
    "shear [1/s]_mean",
    "strain [1/s]_mean",
    "correlation length [m]_mean", 
    "power [W]_mean",
    "work [J]_mean",
    'vector direction [degrees]_mean',
    "Protein Concentration_nM", 
    'time (min)'
]

# Run PCA and save plot (with all conditions and subconditions in the same plot)
qd.plot_pca_expression_piv(data_path, conditions=conditions, subconditions=subconditions, features=features_pca, sigma=1)


In [ ]:

# Example usage
qd.delete_outputs(data_path, conditions, subconditions, output_dirs=None)


In [ ]:
import os
import re
from pathlib import Path
from collections import defaultdict

# Set your directory path
base_dir = Path("../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/060825-ThTr-MTs_MTsFtsZ_FtsZ_FtsZnoDNA/4ultxtl-1ulDNA250ngulorH2O-0p5uldapibeads-1ulFtsZ_/MTs/Rep1/original/")
files = sorted(base_dir.glob("img_*_4x_*.tif"), key=lambda x: x.stat().st_mtime)

# Bundle by timestamp groups of 3 (DAPI, GFP, cy5)
timepoint_groups = []

# Group files into triplets by order (assuming perfect order: [DAPI, GFP, CY5] × N)
if len(files) % 3 != 0:
    raise ValueError(f"Number of files ({len(files)}) is not a multiple of 3. Something's wrong.")

for i in range(0, len(files), 3):
    group = files[i:i+3]
    timepoint_groups.append(group)

if len(timepoint_groups) != 1506:
    raise ValueError(f"Expected 1506 timepoints but found {len(timepoint_groups)}. Check the number of groups.")

# Now rename each triplet
for new_index, group in enumerate(timepoint_groups):
    for file in group:
        match = re.match(r"img_\d+_4x_(\w+).tif", file.name)
        if not match:
            print(f"Skipping: {file.name}")
            continue
        channel = match.group(1)
        new_name = f"img_{new_index:07d}_4x_{channel}.tif"
        new_path = file.with_name(new_name)
        print(f"{file.name} → {new_name}")
        file.rename(new_path)


In [ ]:
import os
from pathlib import Path

# Define your renaming dictionary
conditions_dict = {
    "MTs": "Pos0", 
    "MTs-FtsZ": "Pos1", 
    "FtsZ": "Pos2", 
    "FtsZ-noDNA": "Pos3", 
    "MTs-ZapFtsAT7": "Pos4", 
    "MTs-FtsZ-ZapFtsAT7": "Pos5", 
    "FtsZ-ZapFtsAT7": "Pos6", 
    "FtsZ-noDNA-ZapFtsAT7": "Pos7", 
}

# Path to the parent folder that contains the condition folders
base_dir = Path("../../../../Thomson Lab Dropbox/David Larios/activedrops/ubuntu/060825-ThTr-MTs_MTsFtsZ_FtsZ_FtsZnoDNA/4ultxtl-1ulDNA250ngulorH2O-0p5uldapibeads-1ulFtsZ_1/")  # 👈 change this

# Rename folders
for old_name, new_name in conditions_dict.items():
    old_path = base_dir / old_name
    new_path = base_dir / new_name

    if old_path.exists() and old_path.is_dir():
        print(f"Renaming: {old_name} → {new_name}")
        old_path.rename(new_path)
    else:
        print(f"Skipping: {old_path} not found")
